In [6]:
%%capture
import os
import site
site.addsitedir('/home/schirrmr/.local/lib/python2.7/site-packages/')
site.addsitedir('/usr/lib/pymodules/python2.7/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
%cd /home/schirrmr/braindecode/code/braindecode/
assert 'THEANO_FLAGS' in os.environ
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'
%load_ext autoreload
%autoreload 2
import numpy as np

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

import matplotlib.lines as mlines
import seaborn
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
matplotlib.rcParams['font.size'] = 7
seaborn.set_style('darkgrid')

from braindecode.scripts.print_results import ResultPrinter
from braindecode.csp.print_results import CSPResultPrinter
from braindecode.analysis.pandas_util import (remove_indices_with_same_value,
                                              remove_columns_with_same_value,
                                              dataset_averaged_frame,
                                             load_data_frame,
                                             pairwise_compare_frame)
import logging
log = logging.getLogger()
log.setLevel("DEBUG")

In [3]:
%%javascript
$('body').append('<script src="https://dl.dropboxusercontent.com/u/34637013/work/ipython-notebooks/js-for-table-sorting/jquery.tablesorter.min.js"></script>')
// Sort tables every 5 seconds...
var makeTablesSortable;
makeTablesSortable = function() {
  $('.text_cell_render table').not('.sorted_table').tablesorter().addClass('sorted_table');
}

setInterval(makeTablesSortable, 5000);

<IPython.core.display.Javascript object>

## Basic Model

This is the network structure. In my multiple prediction implementation, when stride is used to half the dimension, the data is moved to the trial axis, not removed as normally would happen. So structure here does not show the whole network, but gives the basic idea.

I don't show batch normalization layers here.

|Layer/Block|#ResBlocks|Units|Filter Size|Nonlinearity/PoolingMode|Output Size|
|-|
|Input|-|22|-|-|1000x1|
|Conv|.|24|(22)x3x1|Relu/Elu|1000x1|
|ResBlocks|2/3|24|3x1|Relu/Elu|1000x1|
|ResBlock|1|48|3x1|Relu/Elu|500x1|
|ResBlocks|2/3|48|3x1|Relu/Elu|500x1|
|ResBlock|1|72|3x1|Relu/Elu|250x1|
|ResBlock|2/3|72|3x1|Relu/Elu|250x1|
|ResBlock|1|72|3x1|Relu/Elu|125x1|
|ResBlock|2/3|72|3x1|Relu/Elu|125x1|
|ResBlock|1|72|3x1|Relu/Elu|63x1|
|ResBlock|2/3|72|3x1|Relu/Elu|63x1|
|ResBlock|1|72|3x1|Relu/Elu|32x1|
|ResBlock|2/3|72|3x1|Relu/Elu|32x1|
|ResBlock|1|72|3x1|Relu/Elu|16x1|
|Pool|-|-|(8/10)x1|Mean|(~7)x1|
|Conv|-|-|1x1|Softmax|7x1|

Then afterwards reshape to get all predictions.

## BCI Competition

Note, Deep5 reaches around 71% on BCI Competition.


In [235]:
df_deep5_bcic = load_data_frame('data/models/paper/bci-competition/cnt/deep4/',
                            shorten_headers=True)


In [245]:
remove_indices_with_same_value(dataset_averaged_frame(df_deep5_bcic)[:2])[1:2]

time  \
                                                                    len   
layers f_time_len double_time_convs drop_prob f_len4 f_len3 f_len2        
cnt_4l 10         -                 -         10     10     10        9   

                                                                             \
                                                                      tmean   
layers f_time_len double_time_convs drop_prob f_len4 f_len3 f_len2            
cnt_4l 10         -                 -         10     10     10     00:18:05   

                                                                             \
                                                                       tstd   
layers f_time_len double_time_convs drop_prob f_len4 f_len3 f_len2            
cnt_4l 10         -                 -         10     10     10     00:05:32   

                                                                    test  \
                                                                    mean   
layers f_time_len double_time_convs drop_prob f_len4 f_len3 f_len2         
cnt_4l 10         -                 -         10     10     10      71.8   

                                                                          \
                                                                     std   
layers f_time_len double_time_convs drop_prob f_len4 f_len3 f_len2         
cnt_4l 10         -                 -         10     10     10      18.8   

                                                                   train       
                                                                    mean  std  
layers f_time_len double_time_convs drop_prob f_len4 f_len3 f_len2             
cnt_4l 10         -                 -         10     10     10      99.9  0.2

I tried out different variants. First an overview overall sorted by test mean accuracy.

* n_l_per_bl => Number of resblocks per residual "superblock" :)
* survival prob is for stocahstic depth. **1** are the same, indicates no stochastic depth.

In [210]:
data_frame = load_data_frame('data/models/paper/bci-competition/cnt/resnet/', params=dict(whisker_percent=None),
                            shorten_headers=True)
data_frame = data_frame[data_frame.nonlinearity != '-']
data_frame = data_frame.drop('final_nonlin', axis=1)
data_frame = data_frame.drop('batch_modifier', axis=1)

data_frame = data_frame.replace(dict(survival_prob={'-': 1}, final_agg={'-': 'pool'}, 
                                 drop_before_pool={'-': 'False'}))
averaged_frame = dataset_averaged_frame(data_frame)

In [211]:
remove_indices_with_same_value(averaged_frame[averaged_frame['time', 'len'] > 1]).sort_values(
    by=[('test', 'mean')], ascending=False)

time  \
                                                                                         len   
bnorm_eps final_p_len drop_before_pool survival_prob n_l_per_bl n_1st_filters final_agg        
0.0001    10          True             1.0           2          48            conv         9   
                      False            1.0           2          64            pool         9   
                      True             1.0           2          64            conv         9   
                                                                48            pool         9   
                      False            1.0           2          48            pool         9   
                                                                64            conv         9   
                                                     1          48            pool         9   
                      True             1.0           2          64            pool         9   
                      False            1.0           2          48            conv         9   
                                                                24            pool         9   
0.0100    10          False            1.0           2          48            pool         9   
0.0001    10          True             1.0           2          24            pool         9   
0.0100    10          True             1.0           2          48            pool         9   
0.0001    10          False            0.5           2          48            pool         9   
                                       1.0           2          24            conv         9   
                      True             1.0           2          24            conv         9   
          9           False            1.0           2          24            pool         9   
          10          False            1.0           3          24            pool         9   
          8           False            1.0           2          24            pool         9   

                                                                                                  \
                                                                                           tmean   
bnorm_eps final_p_len drop_before_pool survival_prob n_l_per_bl n_1st_filters final_agg            
0.0001    10          True             1.0           2          48            conv      01:33:01   
                      False            1.0           2          64            pool      01:49:08   
                      True             1.0           2          64            conv      01:51:05   
                                                                48            pool      01:19:34   
                      False            1.0           2          48            pool      01:27:46   
                                                                64            conv      01:41:30   
                                                     1          48            pool      00:24:30   
                      True             1.0           2          64            pool      02:01:31   
                      False            1.0           2          48            conv      01:16:31   
                                                                24            pool      00:55:52   
0.0100    10          False            1.0           2          48            pool      01:07:09   
0.0001    10          True             1.0           2          24            pool      00:44:49   
0.0100    10          True             1.0           2          48            pool      01:10:02   
0.0001    10          False            0.5           2          48            pool      00:49:05   
                                       1.0           2          24            conv      00:39:04   
                      True             1.0           2          24            conv      00:42:57   
          9           False            1.0           2          24         

## Individual parameters

Here I compare individual parameters when remaining parameters are held fixed.
Note that mean accuracy values can now come from any combination of datasets, i.e. also from just a single dataset and should not be compared to accuracy values over all datasets (from table above).

In [212]:
compare_frame = pairwise_compare_frame(data_frame, with_p_vals=True)

compare_frame.sort_values(by='perm')

,n_exp,val_1,val_2,acc_1,acc_2,diff,std,perm
n_1st_filters,27,48,24,64.6,62.1,-2.5,4.3,0.4
n_l_per_bl,18,2,3,68.4,64.9,-3.5,5.2,0.4
n_1st_filters,27,64,24,64.3,62.1,-2.1,4.5,1.1
nonlinearity,4,elu,relu,79.2,67.0,-12.2,6.3,6.2
final_p_len,13,10,8,62.8,60.6,-2.2,5.0,7.8
final_p_len,9,9,8,61.6,59.1,-2.5,4.3,7.8
final_p_len,9,10,9,63.2,61.6,-1.6,3.1,8.2
bnorm_eps,18,0.0001,0.01,64.2,62.6,-1.6,5.0,10.5
1st_f_len,4,3,11,66.3,59.7,-6.6,6.1,12.5
survival_prob,9,1.0,0.5,64.1,62.1,-2.0,6.9,21.9


## Our Data

Note, deep5 reaches around 90.6% atm

In [247]:
df_deep5_ours = load_data_frame('data/models/paper/ours/cnt/deep4/', params=dict(whisker_percent=None),
                            shorten_headers=True)
dataset_averaged_frame(df_deep5_ours).sort_values(by=('test', 'mean'))[1:2]

time                    test     train     
                len    tmean     tstd  mean std  mean  std
low_cut_off_hz                                            
null             20 01:14:59 00:20:56  90.6   9   100  0.1

## Resnet

In [219]:
data_frame = load_data_frame('data/models/paper/ours/cnt/resnet/', params=dict(whisker_percent=None),
                            shorten_headers=True)
data_frame = data_frame.drop('final_nonlin', axis=1)
data_frame = data_frame.replace(dict(survival_prob={'-': 1}, final_agg={'-': 'pool'}))
#data_frame = data_frame.drop('batch_modifier', axis=1)


In [220]:
dataset_averaged_frame(data_frame).sort_values(by=('test', 'mean'),
                                               ascending=False)

time  \
                                                                                    len   
loss_expression                drop_before_pool survival_prob n_l_per_bl final_agg        
categorical_crossentropy_decay False            1.0           2          conv        20   
safe_tied_neighbours_decay     False            1.0           2          pool        20   
categorical_crossentropy_decay True             1.0           2          pool        20   
                               False            1.0           2          pool        20   
safe_tied_neighbours_decay     False            1.0           1          pool        20   
categorical_crossentropy_decay True             1.0           2          conv        20   
safe_tied_neighbours_decay     False            0.5           2          pool        20   

                                                                                             \
                                                                                      tmean   
loss_expression                drop_before_pool survival_prob n_l_per_bl final_agg            
categorical_crossentropy_decay False            1.0           2          conv      02:23:39   
safe_tied_neighbours_decay     False            1.0           2          pool      02:04:45   
categorical_crossentropy_decay True             1.0           2          pool      02:40:12   
                               False            1.0           2          pool      02:47:28   
safe_tied_neighbours_decay     False            1.0           1          pool      01:20:52   
categorical_crossentropy_decay True             1.0           2          conv      02:20:36   
safe_tied_neighbours_decay     False            0.5           2          pool      02:29:43   

                                                                                             \
                                                                                       tstd   
loss_expression                drop_before_pool survival_prob n_l_per_bl final_agg            
categorical_crossentropy_decay False            1.0           2          conv      00:50:51   
safe_tied_neighbours_decay     False            1.0           2          pool      00:45:25   
categorical_crossentropy_decay True             1.0           2          pool      01:13:22   
                               False            1.0           2          pool      01:35:35   
safe_tied_neighbours_decay     False            1.0           1          pool      00:24:28   
categorical_crossentropy_decay True             1.0           2          conv      01:18:10   
safe_tied_neighbours_decay     False            0.5           2          pool      00:50:19   

                                                                                    test  \
                                                                                    mean   
loss_expression                drop_before_pool survival_prob n_l_per_bl final_agg         
categorical_crossentropy_decay False            1.0           2          conv       87.6   
safe_tied_neighbours_decay     False            1.0           2          pool       87.5   
categorical_crossentropy_decay True             1.0           2          pool       87.0   
                               False            1.0           2          pool       86.8   
safe_tied_neighbours_decay     False            1.0           1          pool       86.8   
categorical_crossentropy_decay True             1.0           2          conv       86.7   
safe_tied_neighbours_decay     False            0.5           2          pool       82.8   

                                                                                         \
                                                                                    std   
loss_expression                drop_before_pool survival_prob n_l_per_bl final_agg        
categorical_crossentropy_decay False            1.0           2          con

#### Individual parameters

In [222]:
compare_frame = pairwise_compare_frame(data_frame, with_p_vals=True)
compare_frame.sort_values(by='perm')

,n_exp,val_1,val_2,acc_1,acc_2,diff,std,perm
survival_prob,20,1.0,0.5,87.5,82.8,-4.7,3.1,0.0
n_l_per_bl,20,2,1,87.5,86.8,-0.8,2.3,10.2
loss_expression,20,safe_tied_neighbours_decay,categorical_crossentropy_decay,87.5,86.8,-0.7,2.7,15.6
drop_before_pool,40,False,True,87.2,86.8,-0.4,3.3,24.4
final_agg,40,conv,pool,87.2,86.9,-0.2,3.3,34.2
